Definition of post_pro() function

In [ ]:
#
# Copyright (C) 2021 by Sonja Filiposka <sonja.filiposka@finki.ukim.mk>
#
# This code is licensed under a Creative Commons Attribution 4.0 International License. (see LICENSE.txt for details)
#
# General Description - this notebook is used to extract data to compute cloud and edge delays from CloudSim logs
#
# It creates 7 types of output files per CloudSim simulation log that contain: 
#    - allocation information of virtualisation entities: 
#       - when allocations are optimal, results are exctracted to <res_alloc_OPT_TIMES_> files
#       - otherwise, allocation information is collected in <res_fail_alloc_detail_> files
#    - migration information of virtualisation entities:
#       - when migrations are optimal, results are extracted to <res_migr_OPT_TIMES_> files
#       - otherwise, migration information is collected in <res_fail_migr_detail_> files
#       - when migration reattempts are successful, results are extracted to <res_reattempt_OK_migr_> files
#       - when reattempts are unsuccessful, information is sent to <res_reattempt_FAIL_migr_> files
#    - energy consumption information is extracted to <res_DCenergy_> files
#
# It also creates 2 files with global data of the whole set of simulations:
#    - one about allocations (<total_resAllocHops.csv>) and 
#    - another about migrations (<total_resMigHops.csv>) 

def post_pro(scenario, vehicles, VMs, path):

    # parameters
    # 1 = $scenario - input scenario format : "A B C D" (A=#MEChosts, B=hostType, C=OptGoal-allocations, D=OptGoal-migrations) i.e. "45 1 sc sc-mad", 
    # 2 = $vehicles - input number of vehicles 
    # 3 = $VMs      - input number of VMs (if omnet simulation time is reduced from SUMO's, this value is different from $vehicles) 
    # 4 = $path     - input files path

    # two output files: resAllocHops.txt resMigHops.txt

    # add only the lines that start with a number
    (noHosts, hostType, ap, mp)  = scenario.split()
    print(noHosts, hostType, ap, mp) 
    # Log format: fogSUMOCloudSim_Log__117_1_sc_sc-mad_2509_initialPositioning-6900.txt_migrations-6900.txt
    fileName = path + "fogSUMOCloudSim_Log__" + noHosts + "_" + hostType + "_" + ap + "_" + mp + "_" + VMs + "_initialPositioning-" + vehicles + ".txt_migrations-" + vehicles + ".txt.txt"
    path = path + "post_pro/"
    optMigrFileTimes  = path + "res_migr_OPT_TIMES_"  + noHosts + "_" + hostType + "_" + ap + "_" + mp + "_" + VMs + "_" + vehicles + ".txt"
    reatFAILMigrFile = path + "res_reattempt_FAIL_migr_"  + noHosts + "_" + hostType + "_" + ap + "_" + mp + "_" + VMs + "_" + vehicles + ".txt"
    reatOKMigrFile  = path + "res_reattempt_OK_migr_"  + noHosts + "_" + hostType + "_" + ap + "_" + mp + "_" + VMs + "_" + vehicles + ".txt"
    FailMigrFileDetail = path + "res_fail_migr_detail_"  + noHosts + "_" + hostType + "_" + ap + "_" + mp + "_" + VMs + "_" + vehicles + ".txt"
    allocOPTFileTimes = path + "res_alloc_OPT_TIMES_"  + noHosts + "_" + hostType + "_" + ap + "_" + mp + "_" + VMs + "_" + vehicles + ".txt"
    FailAllocFileDetail =  path + "res_fail_alloc_detail_" + noHosts + "_" + hostType + "_" + ap + "_" + mp + "_" + VMs + "_" + vehicles + ".txt"
    resDCenergy =  path + "res_DCenergy_"  + noHosts + "_" + hostType + "_" + ap + "_" + mp + "_" + VMs + "_" + vehicles + ".txt"

    print(fileName)
    
    # ENERGY: get energy data
    !grep "Data center's energy" $fileName > temp
    !echo "Time;W*sec" > $resDCenergy
    !awk '{ print $1, $6 }' temp >> temp2
    !cat temp2 >> $resDCenergy
    !rm temp temp2
		
    # ALLOCATIONS ANALYSIS: output files: resFileAllocHops.txt $allocOPTFileTimes
    # get initial allocations
    !grep "Initial placement" $fileName > res_alloc.txt    
    !echo "$noHosts;$hostType;$ap;$mp;$VMs;$vehicles" | tr -s '\r\n' ';' > resFileAllocHops.txt	
    # get the number of lines with optimal allocation
    !grep -o 'optimal=true' res_alloc.txt | wc -l | tr -s '\r\n' ';' >> resFileAllocHops.txt	
    # get the number of lines with non optimal allocations
    !grep -o 'optimal=false' res_alloc.txt | wc -l | tr -s '\r\n' ';' >> resFileAllocHops.txt	
    !echo ""  >> resFileAllocHops.txt					# new line	
    # check optimal allocations: time, car and community
    !echo "Time Vm Comm" > $allocOPTFileTimes
    !grep 'optimal=true' res_alloc.txt	 > temp
    !awk '{print $1, $6, $16}'  temp >  temp2
    !sed 's/\#//g' temp2 >> $allocOPTFileTimes  
    # check failed allocations
    !echo "Time Vm Vm_resources Source_Comm Host Dest_Comm Dest_Comm_decision" > $FailAllocFileDetail
    !grep 'optimal=false' res_alloc.txt > temp
    !awk '{print $1, $6, $7, $10, $13, $16, $17}' temp >  temp2
    !sed 's/\#//g' temp2 >> $FailAllocFileDetail
    !rm temp temp2 res_alloc.txt 

    # MIGRATIONS ANALYSIS: output files: resFileMigHops.txt $FailMigrFileDetail $optMigrFileTimes
    # check reattempted migrations
    !echo "Time Vm " > $reatOKMigrFile
    !grep "reattempt migration: VM *" $fileName > temp
    !awk '{printf ("%.2f %s\n",$1,$5)}' temp > temp2
    !sed 's/\#//g' temp2 >>  $reatOKMigrFile
    # check optimal migrations
    !grep -e "Optimal migration" $fileName > res_opt_migr_opt.txt
    !grep -e "identical" $fileName > res_opt_migr_ident.txt
    !echo "$noHosts;$hostType;$ap;$mp;$VMs;$vehicles" | tr -s '\r\n' ';'  > resFileMigHops.txt
    # check migrations failed
    !echo "Time Vm " > $reatFAILMigrFile
    !grep " reattempt migration failed" $fileName > temp 
    !awk '{printf ("%.2f %s\n",$1,$6)}' temp | sort -g | uniq > temp2
    !sed 's/\#//g' temp2  >> $reatFAILMigrFile
    # get the times of optimal migrations
    !echo "Time Vm Source_Comm Host Dest_Comm " > $optMigrFileTimes
    !awk '{print $1, $6, $9, $12, $15}' res_opt_migr_opt.txt > temp2
    !awk '{print $1, $8, $13, $10, $13}' res_opt_migr_ident.txt >> temp2
    !sed 's/\#//g' temp2  >> $optMigrFileTimes
    # get the number of optimal migrations
    !wc -l temp2 > temp
    !awk '{print $1}' temp | tr -s '\r\n' ';' >> resFileMigHops.txt 	
    # Non optimal migrations
    !echo "Time Vm Source_Comm Dest_Comm" > $FailMigrFileDetail
    !grep  "Attempt to migrate failed" $fileName > temp
    # round attempt times to 2 decimals and remove duplicates
    !awk '{printf ("%.2f %s %s %s\n",$1,$7,$10,$13)}' temp | sort -g | uniq > temp2 #| uniq -u
    !sed 's/\#//g' temp2 >> $FailMigrFileDetail
    # get the number of non optimal migrations
    !wc -l $FailMigrFileDetail > temp
    !awk '{print $1}' temp | tr -s '\r\n' ';' >> resFileMigHops.txt 	
    # get the number of succesful migration reattempts
    !wc -l $reatOKMigrFile > temp	
    !awk '{print $1}' temp | tr -s '\r\n' ';' >> resFileMigHops.txt  
    # get the number of unsuccesful migration reattempts    
    !wc -l $reatFAILMigrFile  > temp	
    !awk '{print $1}' temp | tr -s '\r\n' ';' >> resFileMigHops.txt  	    
    !echo "" >> resFileMigHops.txt  		    # new line	  
    !rm res_opt_migr_opt.txt res_opt_migr_ident.txt temp temp2


time: 110 ms (started: 2021-08-19 06:34:43 +00:00)


Input data : scenarios + vehicles

In [ ]:

    # parameters
    # 1 = $scenario - input scenario format : "A B C D" (A=#MEChosts, B=hostType, C=OptGoal-allocations, D=OptGoal-migrations) i.e. "45 1 sc sc-mad", 
    # 2 = $vehicles - input number of vehicles 
    # 3 = $VMs      - input number of VMs (if omnet simulation time is reduced from SUMO's, this value is different from $vehicles) 
    # 4 = $path     - input files path

    # two output files: resAllocHops.txt resMigHops.txt

# This code launches the first step of CloudSim log information extraction process, calls above function and creates 2 files with global data of the whole set of simulations:
#    - one about allocations (<total_resAllocHops.csv>) and 
#    - another about migrations (<total_resMigHops.csv>) 

# measuring execution time
!pip install ipython-autotime
%load_ext autotime
# define vehicles and scenarios
scenarios = ["45 1 sc sc-mad","63 1 sc sc-mad","81 1 sc sc-mad","99 1 sc sc-mad","117 1 sc sc-mad",
             "45 0 sc sc-mad","63 0 sc sc-mad","81 0 sc sc-mad","99 0 sc sc-mad","117 0 sc sc-mad"]
vehicles = ['4956','4951','4955','5740','5734','5749','6910','6915','6923','8600','8620','8635']
VMs =      ['4256','5074','5013','4966','5808','5858','6042','6187','7004','7455','7463','7486']

duration="10000"
!rm *.csv
#aBorrar2="res_*.txt"

path = "/content/drive/MyDrive/processed_logs/cloudsim_output/log_10000s/"

resFileAllocHops = path + "post_pro/" + "total_resAllocHops.csv"
resFileMigHops = path + "post_pro/" + "total_resMigHops.csv"

!echo "Hosts;Type;Ap;Mp;noVMs;noCars;Optimal;Non_optimal;Reatt_OK;Reatt_FAILED" > $resFileMigHops
!echo "Hosts;Type;Ap;Mp;noVMs;noCars;Optimal;Non_optimal" > $resFileAllocHops


for scen in scenarios:
  i = 0
  for vehi in vehicles:
    !rm *.txt
    post_pro(scen,vehi,VMs[i],path)
    i = i+1
    # update hops
    !cat resFileAllocHops.txt >> $resFileAllocHops
    !cat resFileMigHops.txt >> $resFileMigHops  


rm: cannot remove '*.csv': No such file or directory
rm: cannot remove '*.txt': No such file or directory
45 1 sc sc-mad
/content/drive/MyDrive/processed_logs/cloudsim_output/log_10000s/fogSUMOCloudSim_Log__45_1_sc_sc-mad_4256_initialPositioning-4956.txt_migrations-4956.txt.txt
45 1 sc sc-mad
/content/drive/MyDrive/processed_logs/cloudsim_output/log_10000s/fogSUMOCloudSim_Log__45_1_sc_sc-mad_5074_initialPositioning-4951.txt_migrations-4951.txt.txt
45 1 sc sc-mad
/content/drive/MyDrive/processed_logs/cloudsim_output/log_10000s/fogSUMOCloudSim_Log__45_1_sc_sc-mad_5013_initialPositioning-4955.txt_migrations-4955.txt.txt
45 1 sc sc-mad
/content/drive/MyDrive/processed_logs/cloudsim_output/log_10000s/fogSUMOCloudSim_Log__45_1_sc_sc-mad_4966_initialPositioning-5740.txt_migrations-5740.txt.txt
45 1 sc sc-mad
/content/drive/MyDrive/processed_logs/cloudsim_output/log_10000s/fogSUMOCloudSim_Log__45_1_sc_sc-mad_5808_initialPositioning-5734.txt_migrations-5734.txt.txt
45 1 sc sc-mad
/content/drive